In [1]:
import numpy as np
import pandas as pd
!pip install geopandas
import geopandas as gpd
import folium
from shapely.geometry import Point
from functools import partial
from geopy.geocoders import Nominatim
!pip install opencage
import opencage.geocoder
from shapely import wkt
import warnings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 45.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')
TOM_FILES_DIR = "/content/drive/My Drive/Colab Notebooks/data science- transportation research/"
MAX_FILES_DIR = "/content/drive/My Drive/data science- transportation research/"
FILES_DIR = MAX_FILES_DIR

Mounted at /content/drive


In [ ]:
# df_speed_humps = pd.read_csv(f"{FILES_DIR}with_streets_VZV_Speed Humps.geojson")
# df_speed_limits = gpd.read_file(f"{FILES_DIR}VZV_Speed Limits.geojson")
# df_leading_ped = gpd.read_file(f"{FILES_DIR}VZV_Leading Pedestrian Interval Signals.geojson")
# df_street_imporvements = gpd.read_file(f"{FILES_DIR}VZV_Street Improvement Projects (SIPs) intersections.geojson")
# df_turn_traffic = gpd.read_file(f"{FILES_DIR}NYC Street Centerline (CSCL).geojson")
# df_points_of_interest = gpd.read_file(f"{FILES_DIR}VZV_Turn Traffic Calming.geojson")

In [ ]:
df_speed_humps = df_speed_humps.rename(columns={'new_humps': 'humps_amount','date_insta' : 'end_date'}) 
df_speed_humps_clean = df_speed_humps[["STREET","ST_INDEX","end_date","humps_amount"]]

In [ ]:
df_leading_ped =  pd.read_csv(f"{FILES_DIR}with_streets_VZV_Leading Pedestrian Interval Signals.geojson")

In [ ]:
df_leading_ped_clean = df_leading_ped[["STREET","ST_INDEX","install_da"]]

In [ ]:
df_turn_calming = pd.read_csv(f"{FILES_DIR}with_streets_VZV_Turn Traffic Calming.geojson")

In [ ]:
df_turn_calming_clean = df_turn_calming[["STREET", "ST_INDEX", "completion"]]

In [ ]:
df_speed_limits_new = pd.read_csv(f"{FILES_DIR}with_streets_clean_VZV_Speed Limits.geojson")
df_speed_limits_new = df_speed_limits_new.rename(columns={'postvz_sg': 'is_signed', 'postvz_sl' : 'speed_limit'}) 

In [67]:
df_speed_limits = pd.read_csv(f"{FILES_DIR}with_streets_clean_VZV_Speed Limits.geojson")
df_clean_speed_limits = df_speed_limits.drop(columns=['shape_leng','street','geometry']) 

In [68]:
df_clean_speed_limits= df_clean_speed_limits.rename(columns={'postvz_sg': 'IS_SIGNED', 'postvz_sl' : 'SPEED_LIMIT'}) 

In [69]:
df_clean_speed_limits

,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET,ST_INDEX
0,0,YES,30,MULTILINESTRING ((-73.95140678430934 40.743184...,69179
1,1,YES,25,MULTILINESTRING ((-73.80615939251464 40.786125...,107052
2,2,YES,25,MULTILINESTRING ((-73.8508708750395 40.8312099...,114081
3,3,YES,25,MULTILINESTRING ((-73.73493353117327 40.712983...,83032
4,4,YES,25,MULTILINESTRING ((-74.00805376795842 40.619700...,49378
...,...,...,...,...,...
40068,40068,YES,45,MULTILINESTRING ((-73.99107431903903 40.701605...,118349
40069,40069,YES,25,MULTILINESTRING ((-73.92400298977586 40.708360...,79600
40070,40070,YES,50,MULTILINESTRING ((-73.88386784173494 40.631994...,60787
40071,40071,YES,25,MULTILINESTRING ((-73.96790380981811 40.580242...,115393


In [56]:
df_clean_speed_limits['ST_INDEX'].nunique()

16266

In [53]:
last_rows = df_clean_speed_limits.groupby('ST_INDEX').last()
# select the columns IS_SIGNED and SPEED_LIMIT
result = last_rows[['IS_SIGNED', 'SPEED_LIMIT']]

In [54]:
last_rows

,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET
ST_INDEX,,,,
136,24232,YES,50,MULTILINESTRING ((-73.99332519843287 40.772831...
204,20600,YES,30,MULTILINESTRING ((-74.01442532262277 40.710969...
213,36695,YES,30,MULTILINESTRING ((-74.01352773788118 40.715262...
258,30985,YES,50,MULTILINESTRING ((-73.9934904641407 40.7729289...
263,25025,YES,50,MULTILINESTRING ((-73.9767497487285 40.7959966...
...,...,...,...,...
121109,33105,YES,35,MULTILINESTRING ((-73.86272693403203 40.575824...
121110,28744,YES,30,MULTILINESTRING ((-73.86272693403203 40.575824...
121111,27990,YES,25,MULTILINESTRING ((-73.99488683725635 40.715705...


In [57]:
df_clean_speed_limits = df_clean_speed_limits.loc[df_clean_speed_limits['ST_INDEX'].isin(result.index)]

In [72]:
df_speed_limits_no_dup[df_speed_limits_no_dup['ST_INDEX'] == 85775         ]

,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET,ST_INDEX
10524,10524,YES,20,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
14345,14345,YES,25,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
16326,16326,NO,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
19262,19262,YES,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775


In [71]:
last_rows = df_clean_speed_limits.groupby('ST_INDEX').last().reset_index()
last_rows

,ST_INDEX,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET
0,136,24232,YES,50,MULTILINESTRING ((-73.99332519843287 40.772831...
1,204,20600,YES,30,MULTILINESTRING ((-74.01442532262277 40.710969...
2,213,36695,YES,30,MULTILINESTRING ((-74.01352773788118 40.715262...
3,258,30985,YES,50,MULTILINESTRING ((-73.9934904641407 40.7729289...
4,263,25025,YES,50,MULTILINESTRING ((-73.9767497487285 40.7959966...
...,...,...,...,...,...
16261,121109,33105,YES,35,MULTILINESTRING ((-73.86272693403203 40.575824...
16262,121110,28744,YES,30,MULTILINESTRING ((-73.86272693403203 40.575824...
16263,121111,27990,YES,25,MULTILINESTRING ((-73.99488683725635 40.715705...
16264,121112,21844,YES,35,MULTILINESTRING ((-73.99526551955626 40.715582...


In [75]:
type(last_rows)

pandas.core.frame.DataFrame

In [73]:
df_clean_speed_limits[df_clean_speed_limits['ST_INDEX'] == 85775         ]

,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET,ST_INDEX
10524,10524,YES,20,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
14345,14345,YES,25,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
16326,16326,NO,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
19262,19262,YES,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
21916,21916,YES,25,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
22930,22930,YES,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
23260,23260,YES,25,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
27617,27617,YES,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775


In [74]:
last_rows[last_rows['ST_INDEX'] == 85775         ]

,ST_INDEX,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET
8210,85775,27617,YES,30,MULTILINESTRING ((-73.78306066617594 40.757352...


In [76]:
last_rows['IS_SIGNED'].value_counts()

YES    16186
NO        80
Name: IS_SIGNED, dtype: int64

In [77]:
last_rows

,ST_INDEX,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET
0,136,24232,YES,50,MULTILINESTRING ((-73.99332519843287 40.772831...
1,204,20600,YES,30,MULTILINESTRING ((-74.01442532262277 40.710969...
2,213,36695,YES,30,MULTILINESTRING ((-74.01352773788118 40.715262...
3,258,30985,YES,50,MULTILINESTRING ((-73.9934904641407 40.7729289...
4,263,25025,YES,50,MULTILINESTRING ((-73.9767497487285 40.7959966...
...,...,...,...,...,...
16261,121109,33105,YES,35,MULTILINESTRING ((-73.86272693403203 40.575824...
16262,121110,28744,YES,30,MULTILINESTRING ((-73.86272693403203 40.575824...
16263,121111,27990,YES,25,MULTILINESTRING ((-73.99488683725635 40.715705...
16264,121112,21844,YES,35,MULTILINESTRING ((-73.99526551955626 40.715582...


In [66]:
df_clean_speed_limits

,ST_INDEX,IS_SIGNED,SPEED_LIMIT
0,69179,YES,25
1,107052,YES,30
2,114081,YES,25
3,83032,YES,25
4,49378,YES,25
...,...,...,...
40068,118349,YES,45
40069,79600,YES,25
40070,60787,YES,50
40071,115393,YES,25


In [38]:
df_clean_speed_limits['ST_INDEX'].value_counts()

115114    37
117679    31
119561    30
118368    30
113452    26
          ..
7757       1
74277      1
59246      1
73045      1
32673      1
Name: ST_INDEX, Length: 16266, dtype: int64

In [41]:
df_speed_limits_no_dup = df_clean_speed_limits.drop_duplicates(subset=['ST_INDEX', 'IS_SIGNED', 'SPEED_LIMIT'])

In [43]:
df_speed_limits_no_dup['ST_INDEX'].value_counts()

85775     4
69467     4
120827    4
119229    3
115325    3
         ..
10918     1
17229     1
82746     1
75649     1
32673     1
Name: ST_INDEX, Length: 16266, dtype: int64

In [44]:
df_speed_limits_no_dup[df_speed_limits_no_dup['ST_INDEX'] == 85775         ]

,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET,ST_INDEX
10524,10524,YES,20,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
14345,14345,YES,25,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
16326,16326,NO,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775
19262,19262,YES,30,MULTILINESTRING ((-73.78306066617594 40.757352...,85775


In [45]:
df_speed_limits_no_dup

,Unnamed: 0,IS_SIGNED,SPEED_LIMIT,STREET,ST_INDEX
0,0,YES,30,MULTILINESTRING ((-73.95140678430934 40.743184...,69179
1,1,YES,25,MULTILINESTRING ((-73.80615939251464 40.786125...,107052
2,2,YES,25,MULTILINESTRING ((-73.8508708750395 40.8312099...,114081
3,3,YES,25,MULTILINESTRING ((-73.73493353117327 40.712983...,83032
4,4,YES,25,MULTILINESTRING ((-74.00805376795842 40.619700...,49378
...,...,...,...,...,...
40032,40032,YES,25,MULTILINESTRING ((-73.82965220919704 40.705543...,72499
40034,40034,YES,30,MULTILINESTRING ((-74.19860457894109 40.545679...,76918
40035,40035,YES,25,MULTILINESTRING ((-73.77610411173873 40.759825...,30704
40036,40036,YES,25,MULTILINESTRING ((-73.90112654306378 40.836813...,32673


In [27]:
df

,Unnamed: 0,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,Social Services,Transportation,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546875,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4546876,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4546877,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4546878,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
df_merged = pd.merge(df, df_clean_speed_limits, on=['ST_INDEX'], how='left')
df_merged

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,IS_SIGNED,SPEED_LIMIT,STREET
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969369,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4969370,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4969371,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4969372,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [30]:
df_merged[df_merged['SPEED_LIMIT'].notnull()]


,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,IS_SIGNED,SPEED_LIMIT,STREET
22867,22867,136,MULTILINESTRING ((-73.99332519843287 40.772831...,1,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,19201.0,YES,50.0,MULTILINESTRING ((-73.99332519843287 40.772831...
22868,22867,136,MULTILINESTRING ((-73.99332519843287 40.772831...,1,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23130.0,YES,50.0,MULTILINESTRING ((-73.99332519843287 40.772831...
22869,22867,136,MULTILINESTRING ((-73.99332519843287 40.772831...,1,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24232.0,YES,50.0,MULTILINESTRING ((-73.99332519843287 40.772831...
22870,22868,136,MULTILINESTRING ((-73.99332519843287 40.772831...,1,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,19201.0,YES,50.0,MULTILINESTRING ((-73.99332519843287 40.772831...
22871,22868,136,MULTILINESTRING ((-73.99332519843287 40.772831...,1,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23130.0,YES,50.0,MULTILINESTRING ((-73.99332519843287 40.772831...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4967759,4545265,20491,MULTILINESTRING ((-73.92646866997323 40.752068...,12,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8944.0,YES,25.0,MULTILINESTRING ((-73.92646866997323 40.752068...
4967760,4545266,20491,MULTILINESTRING ((-73.92646866997323 40.752068...,12,2020,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8944.0,YES,25.0,MULTILINESTRING ((-73.92646866997323 40.752068...
4967761,4545267,20491,MULTILINESTRING ((-73.92646866997323 40.752068...,12,2021,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8944.0,YES,25.0,MULTILINESTRING ((-73.92646866997323 40.752068...
4967762,4545268,20491,MULTILINESTRING ((-73.92646866997323 40.752068...,12,2022,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8944.0,YES,25.0,MULTILINESTRING ((-73.92646866997323 40.752068...


In [31]:
df_merged['SPEED_LIMIT'] = df_merged['SPEED_LIMIT'].fillna(25)
df_merged['IS_SIGNED'] = df_merged['IS_SIGNED'].fillna('NO')

In [33]:
df_merged[df_merged['SPEED_LIMIT'].notnull()]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,IS_SIGNED,SPEED_LIMIT,STREET
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969369,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
4969370,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
4969371,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN
4969372,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO,25.0,NaN


In [ ]:
df_speed_humps = pd.read_csv(f"{FILES_DIR}timed_with_streets_clean_VZV_Speed Humps.csv")
df_speed_humps

In [ ]:
df_speed_limits_new_clean = df_speed_limits_new[["STREET", "ST_INDEX", "is_signed", "speed_limit"]]

In [6]:
df_speed_limits = gpd.read_file(f"{FILES_DIR}clean_VZV_Speed Limits.geojson")
df_speed_limits.to_csv(f"{FILES_DIR}clean_VZV_Speed_Limits.csv")

In [10]:
df = pd.read_csv(f"{FILES_DIR}Copy of summons_and_poi_and_collisions_and_streets0.csv")

In [8]:
df_speed_limits = pd.read_csv(f"{FILES_DIR}with_streets_clean_VZV_Speed Limits.geojson")
df_speed_limits

,Unnamed: 0,postvz_sg,postvz_sl,shape_leng,street,geometry,STREET,ST_INDEX
0,0,YES,30,88.900124,PULASKI BRIDGE,MULTILINESTRING ((-73.95140678404174 40.743184...,MULTILINESTRING ((-73.95140678430934 40.743184...,69179
1,1,YES,25,13.295827,FRANCIS LEWIS BOULEVARD,MULTILINESTRING ((-73.80618335195952 40.786172...,MULTILINESTRING ((-73.80615939251464 40.786125...,107052
2,2,YES,25,242.212649,CASTLE HILL AVENUE,MULTILINESTRING ((-73.85071384613964 40.830555...,MULTILINESTRING ((-73.8508708750395 40.8312099...,114081
3,3,YES,25,139.729473,HEMPSTEAD AVENUE,MULTILINESTRING ((-73.734893538745 40.71306819...,MULTILINESTRING ((-73.73493353117327 40.712983...,83032
4,4,YES,25,280.118191,14 AVENUE,MULTILINESTRING ((-74.00522665943119 40.618973...,MULTILINESTRING ((-74.00805376795842 40.619700...,49378
...,...,...,...,...,...,...,...,...
40068,40068,YES,45,200.090879,BROOKLYN QUEENS EXPRESSWAY,MULTILINESTRING ((-73.99039558007716 40.701647...,MULTILINESTRING ((-73.99107431903903 40.701605...,118349
40069,40069,YES,25,222.224399,FLUSHING AVENUE,MULTILINESTRING ((-73.92286218888495 40.708545...,MULTILINESTRING ((-73.92400298977586 40.708360...,79600
40070,40070,YES,50,375.371746,BELT PARKWAY,MULTILINESTRING ((-73.88350466679579 40.632273...,MULTILINESTRING ((-73.88386784173494 40.631994...,60787
40071,40071,YES,25,436.531176,OCEAN PARKWAY,MULTILINESTRING ((-73.96759899350525 40.581556...,MULTILINESTRING ((-73.96790380981811 40.580242...,115393


In [9]:
df_clean_speed_limits = df_speed_limits.drop(columns=['shape_leng','street','geometry']) 
df_clean_speed_limits

,Unnamed: 0,postvz_sg,postvz_sl,STREET,ST_INDEX
0,0,YES,30,MULTILINESTRING ((-73.95140678430934 40.743184...,69179
1,1,YES,25,MULTILINESTRING ((-73.80615939251464 40.786125...,107052
2,2,YES,25,MULTILINESTRING ((-73.8508708750395 40.8312099...,114081
3,3,YES,25,MULTILINESTRING ((-73.73493353117327 40.712983...,83032
4,4,YES,25,MULTILINESTRING ((-74.00805376795842 40.619700...,49378
...,...,...,...,...,...
40068,40068,YES,45,MULTILINESTRING ((-73.99107431903903 40.701605...,118349
40069,40069,YES,25,MULTILINESTRING ((-73.92400298977586 40.708360...,79600
40070,40070,YES,50,MULTILINESTRING ((-73.88386784173494 40.631994...,60787
40071,40071,YES,25,MULTILINESTRING ((-73.96790380981811 40.580242...,115393


In [ ]:
mask = df_speed_humps['YEAR'] < 2012
df_speed_humps.loc[mask, 'YEAR'] = 2012
df_speed_humps.loc[mask, 'MONTH'] = 1

In [ ]:
max_humps = df_speed_humps.groupby(['ST_INDEX', 'YEAR', 'MONTH'])['humps_amount'].max().reset_index()
# merge the max_humps DataFrame back into the original DataFrame
df_no_dup = pd.merge(df_speed_humps, max_humps, on=['ST_INDEX', 'YEAR', 'MONTH'], how='left')
df_no_dup.drop(columns="humps_amount_x",inplace=True)
df_no_dup.rename(columns={'humps_amount_y': 'HUMPS_AMOUNT'}, inplace=True)
df_no_dup

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER,HUMPS_AMOUNT
0,0,MULTILINESTRING ((-73.82297571276175 40.881520...,69497,1,2012,1,4,31,1.0
1,1,MULTILINESTRING ((-73.83886384848017 40.865126...,56208,1,2012,1,4,31,1.0
2,2,MULTILINESTRING ((-73.85865678689959 40.811090...,111543,1,2012,1,4,31,1.0
3,3,MULTILINESTRING ((-73.89715645433532 40.886044...,120706,1,2012,1,4,31,1.0
4,4,MULTILINESTRING ((-73.90821798576806 40.810140...,84836,1,2012,1,2,40,1.0
...,...,...,...,...,...,...,...,...,...
3545,3545,MULTILINESTRING ((-73.87774634675205 40.648407...,115203,5,2016,23,0,21,8.0
3546,3546,MULTILINESTRING ((-73.86919192549185 40.878638...,111451,10,2012,23,1,43,9.0
3547,3547,MULTILINESTRING ((-73.86489746547888 40.888362...,111438,10,2012,23,1,43,9.0
3548,3548,MULTILINESTRING ((-73.74720561385537 40.670364...,29002,9,2014,11,3,37,9.0


In [ ]:
df_no_dup[df_no_dup['humps_amount'] != df_no_dup['max_humps_amount']].count()

Unnamed: 0          51
STREET              51
ST_INDEX            51
humps_amount        51
MONTH               51
YEAR                51
DAY                 51
DAY_OF_WEEK         51
WEEK_NUMBER         51
max_humps_amount    51
dtype: int64

In [ ]:
df_speed_humps[df_speed_humps['ST_INDEX'] == 113785]

,Unnamed: 0,STREET,ST_INDEX,humps_amount,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
1985,1985,MULTILINESTRING ((-73.93462691535909 40.854489...,113785,1.0,6,2015,19,4,25
2306,2306,MULTILINESTRING ((-73.93462691535909 40.854489...,113785,1.0,10,2021,7,3,40
2973,2973,MULTILINESTRING ((-73.93462691535909 40.854489...,113785,2.0,7,2013,12,4,28
3047,3047,MULTILINESTRING ((-73.93462691535909 40.854489...,113785,2.0,9,2015,21,0,39


In [ ]:
duplicates_mask = df_speed_humps.duplicated(subset=['ST_INDEX', 'MONTH','YEAR'])

# Use boolean indexing to select only the duplicated rows
duplicates_df = df_speed_humps[duplicates_mask]
duplicates_df

,Unnamed: 0,STREET,ST_INDEX,humps_amount,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
60,60,MULTILINESTRING ((-73.86832930980042 40.866590...,87470,1.0,1,2012,3,3,1
68,68,MULTILINESTRING ((-73.90010638024903 40.839675...,75611,1.0,1,2012,6,0,2
90,90,MULTILINESTRING ((-73.83490151477424 40.849613...,78062,1.0,1,2012,20,3,47
135,135,MULTILINESTRING ((-73.89003568532674 40.843195...,92229,1.0,1,2012,21,3,25
165,165,MULTILINESTRING ((-73.82941389779815 40.845952...,59435,1.0,1,2012,4,4,22
...,...,...,...,...,...,...,...,...,...
3509,3509,MULTILINESTRING ((-73.90248453533432 40.741316...,115861,4.0,9,2016,27,1,39
3510,3510,MULTILINESTRING ((-73.90248453533432 40.741316...,115861,4.0,9,2016,27,1,39
3511,3511,MULTILINESTRING ((-73.90364763489752 40.738561...,110067,4.0,9,2016,27,1,39
3512,3512,MULTILINESTRING ((-73.90364763489752 40.738561...,110067,4.0,9,2016,27,1,39


In [ ]:
df_speed_humps = df_speed_humps[df_speed_humps['YEAR'] >= 2012]
df_speed_humps

,Unnamed: 0,STREET,ST_INDEX,humps_amount,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
0,0,MULTILINESTRING ((-73.82297571276175 40.881520...,69497,1.0,1,2012,1,4,31
1,1,MULTILINESTRING ((-73.83886384848017 40.865126...,56208,1.0,1,2012,1,4,31
2,2,MULTILINESTRING ((-73.85865678689959 40.811090...,111543,1.0,1,2012,1,4,31
3,3,MULTILINESTRING ((-73.89715645433532 40.886044...,120706,1.0,1,2012,1,4,31
4,4,MULTILINESTRING ((-73.90821798576806 40.810140...,84836,1.0,1,2012,1,2,40
...,...,...,...,...,...,...,...,...,...
3545,3545,MULTILINESTRING ((-73.87774634675205 40.648407...,115203,8.0,5,2016,23,0,21
3546,3546,MULTILINESTRING ((-73.86919192549185 40.878638...,111451,9.0,10,2012,23,1,43
3547,3547,MULTILINESTRING ((-73.86489746547888 40.888362...,111438,9.0,10,2012,23,1,43
3548,3548,MULTILINESTRING ((-73.74720561385537 40.670364...,29002,9.0,9,2014,11,3,37


In [ ]:
df_speed_humps_no_dup = df_no_dup.drop_duplicates(subset=['ST_INDEX', 'MONTH', 'YEAR'])
df_speed_humps_no_dup

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER,HUMPS_AMOUNT
0,0,MULTILINESTRING ((-73.82297571276175 40.881520...,69497,1,2012,1,4,31,1.0
1,1,MULTILINESTRING ((-73.83886384848017 40.865126...,56208,1,2012,1,4,31,1.0
2,2,MULTILINESTRING ((-73.85865678689959 40.811090...,111543,1,2012,1,4,31,1.0
3,3,MULTILINESTRING ((-73.89715645433532 40.886044...,120706,1,2012,1,4,31,1.0
4,4,MULTILINESTRING ((-73.90821798576806 40.810140...,84836,1,2012,1,2,40,1.0
...,...,...,...,...,...,...,...,...,...
3545,3545,MULTILINESTRING ((-73.87774634675205 40.648407...,115203,5,2016,23,0,21,8.0
3546,3546,MULTILINESTRING ((-73.86919192549185 40.878638...,111451,10,2012,23,1,43,9.0
3547,3547,MULTILINESTRING ((-73.86489746547888 40.888362...,111438,10,2012,23,1,43,9.0
3548,3548,MULTILINESTRING ((-73.74720561385537 40.670364...,29002,9,2014,11,3,37,9.0


In [ ]:
df_speed_humps_no_dup['ST_INDEX'].value_counts()

113785    4
107792    3
78478     3
79935     3
83933     3
         ..
109874    1
113285    1
78071     1
53777     1
101132    1
Name: ST_INDEX, Length: 3108, dtype: int64

In [ ]:
df_speed_humps_no_dup = df_no_dup.drop_duplicates(subset=['ST_INDEX', 'MONTH', 'YEAR','HUMPS_AMOUNT'])
df_speed_humps_no_dup

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER,HUMPS_AMOUNT
0,0,MULTILINESTRING ((-73.82297571276175 40.881520...,69497,1,2012,1,4,31,1.0
1,1,MULTILINESTRING ((-73.83886384848017 40.865126...,56208,1,2012,1,4,31,1.0
2,2,MULTILINESTRING ((-73.85865678689959 40.811090...,111543,1,2012,1,4,31,1.0
3,3,MULTILINESTRING ((-73.89715645433532 40.886044...,120706,1,2012,1,4,31,1.0
4,4,MULTILINESTRING ((-73.90821798576806 40.810140...,84836,1,2012,1,2,40,1.0
...,...,...,...,...,...,...,...,...,...
3545,3545,MULTILINESTRING ((-73.87774634675205 40.648407...,115203,5,2016,23,0,21,8.0
3546,3546,MULTILINESTRING ((-73.86919192549185 40.878638...,111451,10,2012,23,1,43,9.0
3547,3547,MULTILINESTRING ((-73.86489746547888 40.888362...,111438,10,2012,23,1,43,9.0
3548,3548,MULTILINESTRING ((-73.74720561385537 40.670364...,29002,9,2014,11,3,37,9.0


In [ ]:
df_merged = pd.merge(df, df_speed_humps_no_dup, on=['ST_INDEX', "YEAR", "MONTH"], how='left')
df_merged

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,HUMPS_AMOUNT
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546875,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4546876,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4546877,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4546878,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_merged['HUMPS_AMOUNT'] = df_merged['HUMPS_AMOUNT'].fillna(0)

In [ ]:
df_merged.sort_values(['ST_INDEX', 'YEAR', 'MONTH'], inplace=True)

In [ ]:
df_merged[df_merged['ST_INDEX'] == 20253 ]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,HUMPS_AMOUNT
4504743,4504743,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,1,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4504755,4504755,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,2,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4504767,4504767,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,3,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4504779,4504779,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,4,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4504791,4504791,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,5,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3189.0,MULTILINESTRING ((-73.91194026136435 40.769010...,3.0,3.0,18.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504838,4504838,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,8,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0
4504850,4504850,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,9,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0
4504862,4504862,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,10,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0
4504874,4504874,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,11,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0


In [ ]:
df['ST_INDEX'].value_counts()

1112     41401
1423     27995
1419     26416
1047     25706
3137     25154
         ...  
10670      144
10671      144
645        144
10674      144
20502      121
Name: ST_INDEX, Length: 20503, dtype: int64

In [ ]:
df_merged[df_merged['HUMPS_AMOUNT'] >1]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,HUMPS_AMOUNT
460731,460731,1144,MULTILINESTRING ((-73.99265431029018 40.737330...,1,2012,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2562.0,MULTILINESTRING ((-73.99265431029018 40.737330...,1.0,2.0,35.0,2.0
460743,460743,1144,MULTILINESTRING ((-73.99265431029018 40.737330...,2,2012,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0
460755,460755,1144,MULTILINESTRING ((-73.99265431029018 40.737330...,3,2012,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0
460767,460767,1144,MULTILINESTRING ((-73.99265431029018 40.737330...,4,2012,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0
460780,460780,1144,MULTILINESTRING ((-73.99265431029018 40.737330...,5,2012,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504838,4504838,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,8,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0
4504850,4504850,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,9,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0
4504862,4504862,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,10,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0
4504874,4504874,20253,MULTILINESTRING ((-73.91194026136435 40.769010...,11,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0


In [ ]:
df_merged.sort_values(['ST_INDEX', 'YEAR', 'MONTH'], inplace=True)


In [ ]:
groups = df_merged.groupby('ST_INDEX')
df_merged['HUMPS_AMOUNT'] = groups['HUMPS_AMOUNT'].cummax()
# fill missing values with the cumulative maximum of the column
#df_merged['HUMPS_AMOUNT'] = df_merged['HUMPS_AMOUNT'].fillna(df_merged['HUMPS_AMOUNT'].cummax())

In [ ]:
df_merged[(df_merged['ST_INDEX'] == 706) &  (df_merged['HUMPS_AMOUNT']>0)]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,HUMPS_AMOUNT
176163,176163,706,MULTILINESTRING ((-74.00697298846217 40.729494...,6,2015,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1593.0,MULTILINESTRING ((-74.00697298846217 40.729494...,10.0,2.0,24.0,1.0
176164,176164,706,MULTILINESTRING ((-74.00697298846217 40.729494...,6,2015,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1593.0,MULTILINESTRING ((-74.00697298846217 40.729494...,10.0,2.0,24.0,1.0
176178,176178,706,MULTILINESTRING ((-74.00697298846217 40.729494...,7,2015,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
176179,176179,706,MULTILINESTRING ((-74.00697298846217 40.729494...,7,2015,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
176191,176191,706,MULTILINESTRING ((-74.00697298846217 40.729494...,8,2015,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176199,176199,706,MULTILINESTRING ((-74.00697298846217 40.729494...,8,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
176211,176211,706,MULTILINESTRING ((-74.00697298846217 40.729494...,9,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
176224,176224,706,MULTILINESTRING ((-74.00697298846217 40.729494...,10,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
176236,176236,706,MULTILINESTRING ((-74.00697298846217 40.729494...,11,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:

df_traffic_calming = pd.read_csv(f"{FILES_DIR}timed_with_streets_VZV_Turn Traffic Calming.geojson.csv")
df_traffic_calming

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
0,0,MULTILINESTRING ((-73.9527599491435 40.8050304...,19193,12,2021,15,2,50
1,1,MULTILINESTRING ((-73.86714843148845 40.663916...,66624,10,2021,8,4,40
2,2,MULTILINESTRING ((-73.93982667856882 40.838497...,24867,12,2019,19,3,51
3,3,MULTILINESTRING ((-73.82447123615576 40.794378...,31331,4,2021,22,3,16
4,4,MULTILINESTRING ((-73.99404155268417 40.720320...,105431,8,2020,1,5,31
...,...,...,...,...,...,...,...,...
809,809,MULTILINESTRING ((-73.97241481651272 40.632993...,117005,12,2022,21,2,51
810,810,MULTILINESTRING ((-73.99386586462516 40.633347...,38455,12,2022,23,4,51
811,811,MULTILINESTRING ((-73.96465507445552 40.774836...,110304,12,2022,27,1,52
812,812,MULTILINESTRING ((-73.97597593768323 40.786368...,20561,12,2022,27,1,52


In [ ]:
df_traffic_calming = df_traffic_calming.drop(columns=["DAY", "DAY_OF_WEEK", "WEEK_NUMBER"])

In [ ]:
df_traffic_calming = df_traffic_calming[df_traffic_calming['YEAR'] >= 2012]
df_traffic_calming = df_traffic_calming.drop_duplicates(subset=['ST_INDEX', 'MONTH', 'YEAR'])
df_traffic_calming

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR
0,0,MULTILINESTRING ((-73.9527599491435 40.8050304...,19193,12,2021
1,1,MULTILINESTRING ((-73.86714843148845 40.663916...,66624,10,2021
2,2,MULTILINESTRING ((-73.93982667856882 40.838497...,24867,12,2019
3,3,MULTILINESTRING ((-73.82447123615576 40.794378...,31331,4,2021
4,4,MULTILINESTRING ((-73.99404155268417 40.720320...,105431,8,2020
...,...,...,...,...,...
809,809,MULTILINESTRING ((-73.97241481651272 40.632993...,117005,12,2022
810,810,MULTILINESTRING ((-73.99386586462516 40.633347...,38455,12,2022
811,811,MULTILINESTRING ((-73.96465507445552 40.774836...,110304,12,2022
812,812,MULTILINESTRING ((-73.97597593768323 40.786368...,20561,12,2022


In [ ]:
df_merged = pd.merge(df, df_traffic_calming, on=['ST_INDEX', "YEAR", "MONTH"], how='left')
df_merged["new_binary_column"] = df_merged['STREET'].apply(lambda x: 1 if pd.notnull(x) else 0)
df_merged = df_merged.drop(columns=["STREET"])
df_merged

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,new_binary_column
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546875,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4546876,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4546877,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4546878,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0


In [ ]:
df_merged[df_merged['new_binary_column'] == 1 ]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,new_binary_column
154205,154205,589,MULTILINESTRING ((-74.006298417107 40.72254950...,12,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,471.0,1
227124,227124,845,MULTILINESTRING ((-74.00532369793707 40.718273...,12,2018,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,279.0,1
234593,234593,883,MULTILINESTRING ((-74.0083783124243 40.7165874...,12,2016,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,1
234594,234594,883,MULTILINESTRING ((-74.0083783124243 40.7165874...,12,2016,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,1
301578,301578,959,MULTILINESTRING ((-73.98957565789482 40.741548...,12,2018,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,395.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4380332,4380332,19566,MULTILINESTRING ((-73.81525074758389 40.753138...,4,2021,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,371.0,1
4395204,4395204,19612,MULTILINESTRING ((-73.76994195288674 40.761437...,12,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,1
4427391,4427391,19824,MULTILINESTRING ((-73.76716451639783 40.755657...,1,2022,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,593.0,1
4476688,4476688,20076,MULTILINESTRING ((-73.92842541023079 40.763415...,12,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,528.0,1


In [ ]:
df_merged = df_merged.sort_values(['ST_INDEX', 'YEAR', 'MONTH'])
groups = df_merged.groupby('ST_INDEX')
df_merged['new_binary_column'] = groups['new_binary_column'].cummax()

In [ ]:
df_merged[df_merged['new_binary_column'] == 1 ]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,new_binary_column
154205,154205,589,MULTILINESTRING ((-74.006298417107 40.72254950...,12,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,471.0,1
153966,153966,589,MULTILINESTRING ((-74.006298417107 40.72254950...,1,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
153981,153981,589,MULTILINESTRING ((-74.006298417107 40.72254950...,2,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
153998,153998,589,MULTILINESTRING ((-74.006298417107 40.72254950...,3,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
154011,154011,589,MULTILINESTRING ((-74.006298417107 40.72254950...,4,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546563,4546563,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,8,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
4546575,4546575,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,9,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
4546587,4546587,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,10,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1
4546599,4546599,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,11,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1


In [ ]:
df_intersections_improvement = pd.read_csv(f"{FILES_DIR}timed_with_streets_clean_VZV_Street Improvement Projects (SIPs) intersections.csv")
df_intersections_improvement

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
0,0,MULTILINESTRING ((-73.86237157609695 40.865530...,98595,8,2018,29,2,35
1,1,MULTILINESTRING ((-74.01102885538923 40.640975...,45834,12,2022,28,2,52
2,2,MULTILINESTRING ((-73.9799935381488 40.6520326...,78593,12,2018,19,2,51
3,3,MULTILINESTRING ((-73.90715633910129 40.676097...,120268,11,2012,15,3,46
4,4,MULTILINESTRING ((-73.95841539467996 40.820625...,118342,7,2021,29,3,30
...,...,...,...,...,...,...,...,...
287,287,MULTILINESTRING ((-73.94084495843477 40.620226...,118624,6,2017,9,4,23
288,288,MULTILINESTRING ((-73.97870368147461 40.683744...,52819,12,2011,6,1,49
289,289,MULTILINESTRING ((-73.868288724959 40.86791055...,50220,1,2021,12,1,2
290,290,MULTILINESTRING ((-73.90558146454062 40.905892...,113007,9,2017,29,4,39


In [ ]:
df_intersections_improvement =df_intersections_improvement.drop(columns=["DAY", "DAY_OF_WEEK", "WEEK_NUMBER"])
df_intersections_improvement = df_intersections_improvement[df_intersections_improvement['YEAR'] >= 2012]

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR
0,0,MULTILINESTRING ((-73.86237157609695 40.865530...,98595,8,2018
1,1,MULTILINESTRING ((-74.01102885538923 40.640975...,45834,12,2022
2,2,MULTILINESTRING ((-73.9799935381488 40.6520326...,78593,12,2018
3,3,MULTILINESTRING ((-73.90715633910129 40.676097...,120268,11,2012
4,4,MULTILINESTRING ((-73.95841539467996 40.820625...,118342,7,2021
...,...,...,...,...,...
286,286,MULTILINESTRING ((-73.96612747295728 40.762179...,20629,5,2014
287,287,MULTILINESTRING ((-73.94084495843477 40.620226...,118624,6,2017
289,289,MULTILINESTRING ((-73.868288724959 40.86791055...,50220,1,2021
290,290,MULTILINESTRING ((-73.90558146454062 40.905892...,113007,9,2017


In [ ]:
df_intersections_improvement = df_intersections_improvement.drop_duplicates(subset=['ST_INDEX', 'MONTH', 'YEAR'])
df_intersections_improvement

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR
0,0,MULTILINESTRING ((-73.86237157609695 40.865530...,98595,8,2018
1,1,MULTILINESTRING ((-74.01102885538923 40.640975...,45834,12,2022
2,2,MULTILINESTRING ((-73.9799935381488 40.6520326...,78593,12,2018
3,3,MULTILINESTRING ((-73.90715633910129 40.676097...,120268,11,2012
4,4,MULTILINESTRING ((-73.95841539467996 40.820625...,118342,7,2021
...,...,...,...,...,...
286,286,MULTILINESTRING ((-73.96612747295728 40.762179...,20629,5,2014
287,287,MULTILINESTRING ((-73.94084495843477 40.620226...,118624,6,2017
289,289,MULTILINESTRING ((-73.868288724959 40.86791055...,50220,1,2021
290,290,MULTILINESTRING ((-73.90558146454062 40.905892...,113007,9,2017


In [ ]:
df_merged_2 = pd.merge(df, df_intersections_improvement, on=['ST_INDEX', "YEAR", "MONTH"], how='left')
df_merged_2

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546875,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4546876,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4546877,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4546878,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:
df_merged_2['INCLUDED_LEADING_PEDESTRIAN_INTERVAL_SIGNALS'] = df_merged_2['STREET'].apply(lambda x: 1 if pd.notnull(x) else 0)
df_merged_2 = df_merged_2.drop(columns=["STREET"])
df_merged_2

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,INCLUDED_LEADING_PEDESTRIAN_INTERVAL_SIGNALS
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546875,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4546876,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4546877,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4546878,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0


In [ ]:
df_merged_2 = df_merged_2.sort_values(['ST_INDEX', 'YEAR', 'MONTH'])
groups = df_merged_2.groupby('ST_INDEX')
df_merged_2['INCLUDED_LEADING_PEDESTRIAN_INTERVAL_SIGNALS'] = groups['INCLUDED_LEADING_PEDESTRIAN_INTERVAL_SIGNALS'].cummax()

In [ ]:

df_merged_2[df_merged_2['INCLUDED_LEADING_PEDESTRIAN_INTERVAL_SIGNALS'] == 1 ]

KeyboardInterrupt: ignored

In [ ]:
df =df.drop(columns=["DAY", "DAY_OF_WEEK", "WEEK_NUMBER"])
df = df[df['YEAR'] >= 2012]
df = df.drop_duplicates(subset=['ST_INDEX', 'MONTH', 'YEAR'])
df_merged = pd.merge(main_df, df, on=['ST_INDEX', "YEAR", "MONTH"], how='left')
df_merged['INCLUDED_LEADING_PEDESTRIAN_INTERVAL_SIGNALS'] = df_merged['STREET'].apply(lambda x: 1 if pd.notnull(x) else 0)
df_merged = df_merged.drop(columns=["STREET"])
df_merged.to_csv(f"{dir_path}\\leading_pedestrian_and_{main_file}")

In [ ]:
df_pedes = pd.read_csv(f"{FILES_DIR}timed_with_streets_VZV_Leading Pedestrian Interval Signals.geojson.csv")
df_pedes

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
0,0,MULTILINESTRING ((-74.02200366499811 40.618938...,33197,6.0,2020.0,4.0,3.0,23.0
1,1,MULTILINESTRING ((-74.01984163966091 40.617643...,106098,6.0,2020.0,4.0,3.0,23.0
2,2,MULTILINESTRING ((-74.01611824015598 40.615384...,105712,6.0,2020.0,4.0,3.0,23.0
3,3,MULTILINESTRING ((-74.01611824015598 40.615384...,105712,6.0,2020.0,4.0,3.0,23.0
4,4,MULTILINESTRING ((-74.01171201555734 40.612735...,38212,6.0,2020.0,4.0,3.0,23.0
...,...,...,...,...,...,...,...,...
5944,5944,MULTILINESTRING ((-73.89588135872874 40.820992...,78920,2.0,2023.0,24.0,4.0,8.0
5945,5945,MULTILINESTRING ((-73.83355838813621 40.687803...,29416,2.0,2023.0,24.0,4.0,8.0
5946,5946,MULTILINESTRING ((-73.81524272749512 40.704202...,10965,2.0,2023.0,24.0,4.0,8.0
5947,5947,MULTILINESTRING ((-73.88559269568417 40.738326...,28490,2.0,2023.0,28.0,1.0,9.0


In [ ]:
duplicates_mask = df_pedes.duplicated(subset=['ST_INDEX', 'MONTH','YEAR'])

# Use boolean indexing to select only the duplicated rows
duplicates_df = df_pedes[duplicates_mask]
duplicates_df

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
3,3,MULTILINESTRING ((-74.01611824015598 40.615384...,105712,6.0,2020.0,4.0,3.0,23.0
128,128,MULTILINESTRING ((-73.98271968667424 40.678087...,116654,6.0,2020.0,19.0,4.0,25.0
287,287,MULTILINESTRING ((-73.90895990309998 40.830226...,119713,5.0,2020.0,21.0,3.0,21.0
291,291,MULTILINESTRING ((-73.92819257918781 40.660177...,116513,5.0,2020.0,21.0,3.0,21.0
363,363,MULTILINESTRING ((-73.92986716555932 40.865956...,111690,2.0,2020.0,18.0,1.0,8.0
...,...,...,...,...,...,...,...,...
5619,5619,MULTILINESTRING ((-74.14085978469886 40.633338...,78356,8.0,2020.0,27.0,3.0,35.0
5771,5771,MULTILINESTRING ((-73.94642266064318 40.624385...,113075,7.0,2021.0,23.0,4.0,29.0
5860,5860,MULTILINESTRING ((-73.88349099858058 40.657591...,66980,7.0,2021.0,16.0,4.0,28.0
5861,5861,MULTILINESTRING ((-73.80825843364198 40.686963...,16546,12.0,2021.0,20.0,0.0,51.0


In [ ]:
df_pedes = df_pedes.drop_duplicates(subset=['ST_INDEX', 'MONTH','YEAR'])
df_pedes

,Unnamed: 0,STREET,ST_INDEX,MONTH,YEAR,DAY,DAY_OF_WEEK,WEEK_NUMBER
0,0,MULTILINESTRING ((-74.02200366499811 40.618938...,33197,6.0,2020.0,4.0,3.0,23.0
1,1,MULTILINESTRING ((-74.01984163966091 40.617643...,106098,6.0,2020.0,4.0,3.0,23.0
2,2,MULTILINESTRING ((-74.01611824015598 40.615384...,105712,6.0,2020.0,4.0,3.0,23.0
4,4,MULTILINESTRING ((-74.01171201555734 40.612735...,38212,6.0,2020.0,4.0,3.0,23.0
5,5,MULTILINESTRING ((-73.99825418891973 40.604608...,50882,6.0,2020.0,4.0,3.0,23.0
...,...,...,...,...,...,...,...,...
5944,5944,MULTILINESTRING ((-73.89588135872874 40.820992...,78920,2.0,2023.0,24.0,4.0,8.0
5945,5945,MULTILINESTRING ((-73.83355838813621 40.687803...,29416,2.0,2023.0,24.0,4.0,8.0
5946,5946,MULTILINESTRING ((-73.81524272749512 40.704202...,10965,2.0,2023.0,24.0,4.0,8.0
5947,5947,MULTILINESTRING ((-73.88559269568417 40.738326...,28490,2.0,2023.0,28.0,1.0,9.0


In [ ]:
num_unique = df_pedes['STREET'].nunique()
print(num_unique)

5588


In [ ]:
relevant_pd = df_pedes[df_pedes['YEAR'] >=2012]
num_unique = relevant_pd['STREET'].nunique()
print(num_unique)
relevant_pd['ST_INDEX']

5404


0        33197
1       106098
2       105712
4        38212
5        50882
         ...  
5944     78920
5945     29416
5946     10965
5947     28490
5948     54962
Name: ST_INDEX, Length: 5631, dtype: int64

In [ ]:
df = pd.read_csv(f"{FILES_DIR}Copy of summons_and_poi_and_collisions_and_streets0.csv")
df

,Unnamed: 0,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,Social Services,Transportation,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546875,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4546876,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4546877,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4546878,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
all_included = relevant_pd['ST_INDEX'].isin(df['ST_INDEX']).all()
print(all_included)

True


In [ ]:
df[df['ST_INDEX'] == 10965]

,Unnamed: 0,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,Social Services,Transportation,Commercial,Government,Religious Institution,Health Services,Public Safety,Water,Miscellaneous,SUMMONS
2753233,2753233,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,1,2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2753234,2753234,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,1,2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2753235,2753235,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,1,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2753236,2753236,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,1,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2753237,2753237,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,1,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753648,2753648,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,12,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
2753649,2753649,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,12,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
2753650,2753650,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,12,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
2753651,2753651,10965,MULTILINESTRING ((-73.81524272749512 40.704202...,12,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0


In [ ]:
df_merged = pd.merge(df, df_pedes, on=['ST_INDEX', "YEAR", "MONTH"], how='left')
#df_merged = df_merged.drop(columns="STREET")
df_merged


,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,1,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2013,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,3,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2014,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,4,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2015,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546875,4546875,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4546876,4546876,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4546877,4546877,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4546878,4546878,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,10,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_merged[df_merged['STREET'].notna()] 

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Health Services,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER
13428,13428,80,MULTILINESTRING ((-74.01150567239777 40.702958...,8,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4338.0,MULTILINESTRING ((-74.01150567239777 40.702958...,17.0,2.0,33.0
132157,132157,529,MULTILINESTRING ((-74.003259551152 40.72037709...,9,2019,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,691.0,MULTILINESTRING ((-74.003259551152 40.72037709...,19.0,3.0,38.0
152707,152707,584,MULTILINESTRING ((-74.0101824041449 40.7201150...,3,2022,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5082.0,MULTILINESTRING ((-74.0101824041449 40.7201150...,18.0,4.0,11.0
154210,154210,589,MULTILINESTRING ((-74.006298417107 40.72254950...,12,2021,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,4954.0,MULTILINESTRING ((-74.006298417107 40.72254950...,9.0,3.0,49.0
154211,154211,589,MULTILINESTRING ((-74.006298417107 40.72254950...,12,2021,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,4954.0,MULTILINESTRING ((-74.006298417107 40.72254950...,9.0,3.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4541178,4541178,20465,MULTILINESTRING ((-73.89935599671688 40.710323...,4,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2075.0,MULTILINESTRING ((-73.89935599671688 40.710323...,3.0,1.0,14.0
4542683,4542683,20475,MULTILINESTRING ((-73.91766950158839 40.755180...,5,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2037.0,MULTILINESTRING ((-73.91766950158839 40.755180...,12.0,5.0,19.0
4544131,4544131,20484,MULTILINESTRING ((-73.90525330085518 40.775692...,9,2017,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2518.0,MULTILINESTRING ((-73.90525330085518 40.775692...,25.0,0.0,39.0
4545180,4545180,20491,MULTILINESTRING ((-73.92646866997323 40.752068...,5,2018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2030.0,MULTILINESTRING ((-73.92646866997323 40.752068...,12.0,5.0,19.0


In [ ]:
df_merged['has_value'] = df_merged['STREET'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [ ]:
df_merged = df_merged.sort_values(['ST_INDEX', 'YEAR', 'MONTH'])
df_merged

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,has_value
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
19,19,0,MULTILINESTRING ((-74.01793061274537 40.706174...,2,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
35,35,0,MULTILINESTRING ((-74.01793061274537 40.706174...,3,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
54,54,0,MULTILINESTRING ((-74.01793061274537 40.706174...,4,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
68,68,0,MULTILINESTRING ((-74.01793061274537 40.706174...,5,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546822,4546822,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,5,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4546834,4546834,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,6,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4546846,4546846,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,7,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4546858,4546858,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,8,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
groups = df_merged.groupby('ST_INDEX')
df_merged['has_value'] = groups['has_value'].cummax()

groups['has_value']

In [ ]:
df_merged['has_value'] = groups['has_value'].cummax()
df_merged

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,has_value
0,0,0,MULTILINESTRING ((-74.01793061274537 40.706174...,1,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
19,19,0,MULTILINESTRING ((-74.01793061274537 40.706174...,2,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
35,35,0,MULTILINESTRING ((-74.01793061274537 40.706174...,3,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
54,54,0,MULTILINESTRING ((-74.01793061274537 40.706174...,4,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
68,68,0,MULTILINESTRING ((-74.01793061274537 40.706174...,5,2012,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546822,4546822,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,5,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4546834,4546834,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,6,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4546846,4546846,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,7,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4546858,4546858,20502,MULTILINESTRING ((-73.92216697479408 40.773546...,8,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
df_merged[df_merged['has_value'] == 1]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,has_value
13428,13428,80,MULTILINESTRING ((-74.01150567239777 40.702958...,8,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4338.0,MULTILINESTRING ((-74.01150567239777 40.702958...,17.0,2.0,33.0,1
13440,13440,80,MULTILINESTRING ((-74.01150567239777 40.702958...,9,2016,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13452,13452,80,MULTILINESTRING ((-74.01150567239777 40.702958...,10,2016,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13464,13464,80,MULTILINESTRING ((-74.01150567239777 40.702958...,11,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13477,13477,80,MULTILINESTRING ((-74.01150567239777 40.702958...,12,2016,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546563,4546563,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,8,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
4546575,4546575,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,9,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
4546587,4546587,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,10,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
4546599,4546599,20500,MULTILINESTRING ((-73.92398142938228 40.774615...,11,2023,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
df_merged[(df_merged['ST_INDEX'] ==80) & (df_merged['has_value'] == 1)]

,Unnamed: 0_x,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,...,Public Safety,Water,Miscellaneous,SUMMONS,Unnamed: 0_y,STREET,DAY,DAY_OF_WEEK,WEEK_NUMBER,has_value
13428,13428,80,MULTILINESTRING ((-74.01150567239777 40.702958...,8,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4338.0,MULTILINESTRING ((-74.01150567239777 40.702958...,17.0,2.0,33.0,1
13440,13440,80,MULTILINESTRING ((-74.01150567239777 40.702958...,9,2016,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13452,13452,80,MULTILINESTRING ((-74.01150567239777 40.702958...,10,2016,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13464,13464,80,MULTILINESTRING ((-74.01150567239777 40.702958...,11,2016,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13477,13477,80,MULTILINESTRING ((-74.01150567239777 40.702958...,12,2016,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13435,13435,80,MULTILINESTRING ((-74.01150567239777 40.702958...,8,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13447,13447,80,MULTILINESTRING ((-74.01150567239777 40.702958...,9,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13459,13459,80,MULTILINESTRING ((-74.01150567239777 40.702958...,10,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
13471,13471,80,MULTILINESTRING ((-74.01150567239777 40.702958...,11,2023,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
#x=df_pedes.groupby([ "ST_INDEX","YEAR","MONTH"])
#df_pedes['ST_INDEX'].value_counts()
male_rows = df_pedes[df_pedes['ST_INDEX'] == 98885]
print(male_rows)

      Unnamed: 0                                             STREET  ST_INDEX  \
3138        3138  MULTILINESTRING ((-73.95318575550459 40.817600...     98885   
4454        4454  MULTILINESTRING ((-73.95318575550459 40.817600...     98885   
4622        4622  MULTILINESTRING ((-73.95318575550459 40.817600...     98885   

      MONTH    YEAR   DAY  DAY_OF_WEEK  WEEK_NUMBER  
3138    7.0  2016.0  18.0          0.0         29.0  
4454    7.0  2016.0  18.0          0.0         29.0  
4622    7.0  2016.0  18.0          0.0         29.0  


In [ ]:
df_speed_limits_new_clean.to_csv(f"{FILES_DIR}with_streets_clean_VZV_Speed_Limits")

In [ ]:
df_street_imporvements = pd.read_csv(f"{FILES_DIR}with_streets_VZV_Street Improvement Projects (SIPs) intersections.geojson")
df_street_imporvements_clean = df_street_imporvements[["STREET","ST_INDEX","end_date"]]

In [ ]:
df_street_imporvements_clean.to_csv(f"{FILES_DIR}with_streets_clean_VZV_Street Improvement Projects (SIPs) intersections")

In [ ]:
df_speed_limits = gpd.read_file(f"{FILES_DIR}VZV_Speed Limits.geojson")

In [ ]:
df_speed_limits

,postvz_sg,postvz_sl,shape_leng,street,geometry
0,NO,25,303.41621668,HAZEN STREET,"MULTILINESTRING ((-73.88401 40.79389, -73.8840..."
1,YES,30,88.900124317,PULASKI BRIDGE,"MULTILINESTRING ((-73.95141 40.74318, -73.9514..."
2,NO,25,185.667966712,GRAND CONCOURSE,"MULTILINESTRING ((-73.90208 40.85462, -73.9017..."
3,YES,25,13.2958267314,FRANCIS LEWIS BOULEVARD,"MULTILINESTRING ((-73.80618 40.78617, -73.8061..."
4,NO,25,163.651317782,CROSS BRONX EXPRESSWAY SR SOUTH,"MULTILINESTRING ((-73.82572 40.82468, -73.8251..."
...,...,...,...,...,...
137474,NO,25,281.836010347,AVENUE U,"MULTILINESTRING ((-73.92332 40.60920, -73.9225..."
137475,NO,25,45.1215689613,54 STREET,"MULTILINESTRING ((-74.02162 40.64820, -74.0214..."
137476,NO,25,99.1272043455,BUTLER PLACE,"MULTILINESTRING ((-73.84319 40.83817, -73.8428..."
137477,NO,0,298.91629534,JACKIE ROBINSON PKWY ET 3 EB,"MULTILINESTRING ((-73.88578 40.69130, -73.8847..."


In [ ]:
(df_speed_limits['postvz_sl'].astype(int) < 25).sum()


8228

In [ ]:
df_speed_limits['postvz_sg'].value_counts()

NO     97632
YES    39809
No        38
Name: postvz_sg, dtype: int64

In [ ]:
test_df = df_speed_limits[df_speed_limits['postvz_sl'].astype(int) == 0]
test_df['postvz_sg'].value_counts()

NO    5218
Name: postvz_sg, dtype: int64

In [ ]:
no_zero_df = df_speed_limits[df_speed_limits['postvz_sl'].astype(int) != 0]

ValueError: ignored

In [ ]:
no_zero_df

,postvz_sg,postvz_sl,shape_leng,street,geometry
0,NO,25,303.41621668,HAZEN STREET,"MULTILINESTRING ((-73.88401 40.79389, -73.8840..."
1,YES,30,88.900124317,PULASKI BRIDGE,"MULTILINESTRING ((-73.95141 40.74318, -73.9514..."
2,NO,25,185.667966712,GRAND CONCOURSE,"MULTILINESTRING ((-73.90208 40.85462, -73.9017..."
3,YES,25,13.2958267314,FRANCIS LEWIS BOULEVARD,"MULTILINESTRING ((-73.80618 40.78617, -73.8061..."
4,NO,25,163.651317782,CROSS BRONX EXPRESSWAY SR SOUTH,"MULTILINESTRING ((-73.82572 40.82468, -73.8251..."
...,...,...,...,...,...
137472,YES,40,92.0135281342,F D R DRIVE,"MULTILINESTRING ((-73.92920 40.79699, -73.9291..."
137473,NO,25,386.546488289,HUGHES AVENUE,"MULTILINESTRING ((-73.88687 40.85520, -73.8861..."
137474,NO,25,281.836010347,AVENUE U,"MULTILINESTRING ((-73.92332 40.60920, -73.9225..."
137475,NO,25,45.1215689613,54 STREET,"MULTILINESTRING ((-74.02162 40.64820, -74.0214..."


In [ ]:
not_zero_not_25_df = no_zero_df[df_speed_limits['postvz_sl'].astype(int) != 25 ]

/usr/local/lib/python3.9/dist-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [ ]:
not_zero_not_25_df

,postvz_sg,postvz_sl,shape_leng,street,geometry
1,YES,30,88.900124317,PULASKI BRIDGE,"MULTILINESTRING ((-73.95141 40.74318, -73.9514..."
29,YES,35,36.6547910825,RICHMOND AVENUE,"MULTILINESTRING ((-74.16313 40.59250, -74.1631..."
44,YES,50,534.405029025,NEW ENGLAND THRUWAY,"MULTILINESTRING ((-73.82842 40.88275, -73.8274..."
45,YES,50,201.352616291,CROSS BRONX EXPRESSWAY,"MULTILINESTRING ((-73.90409 40.84477, -73.9034..."
49,YES,50,22.6047446747,CROSS BRONX EXPRESSWAY,"MULTILINESTRING ((-73.88179 40.83799, -73.8817..."
...,...,...,...,...,...
137448,YES,40,18.0298917335,F D R DRIVE,"MULTILINESTRING ((-73.99886 40.70846, -73.9988..."
137454,YES,40,221.749361966,PELHAM PARKWAY,"MULTILINESTRING ((-73.82984 40.85613, -73.8290..."
137455,YES,45,200.090878509,BROOKLYN QUEENS EXPRESSWAY,"MULTILINESTRING ((-73.99040 40.70165, -73.9896..."
137465,YES,50,375.371745688,BELT PARKWAY,"MULTILINESTRING ((-73.88350 40.63227, -73.8835..."


In [ ]:
df_testing = df_speed_limits[(df_speed_limits['postvz_sg'] == 'NO') | (df_speed_limits['postvz_sg'] == 'No')]


In [ ]:
df_testing['postvz_sl'].value_counts()

25    92188
0      5218
30      175
35       40
50       26
40       14
20        8
45        1
Name: postvz_sl, dtype: int64

In [ ]:
clean_speed_limits_df = no_zero_df[((df_speed_limits['postvz_sl'].astype(int) == 25) & (df_speed_limits['postvz_sg'] == 'YES')) | (df_speed_limits['postvz_sl'].astype(int) != 25)]

/usr/local/lib/python3.9/dist-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [ ]:
clean_speed_limits_df.to_file(f"{FILES_DIR}clean_VZV_Speed Limits.geojson", driver='GeoJSON')

In [ ]:
df_read = gpd.read_file(f"{FILES_DIR}clean_VZV_Speed Limits.geojson")
df_read

,postvz_sg,postvz_sl,shape_leng,street,geometry
0,YES,30,88.900124317,PULASKI BRIDGE,"MULTILINESTRING ((-73.95141 40.74318, -73.9514..."
1,YES,25,13.2958267314,FRANCIS LEWIS BOULEVARD,"MULTILINESTRING ((-73.80618 40.78617, -73.8061..."
2,YES,25,242.212648614,CASTLE HILL AVENUE,"MULTILINESTRING ((-73.85071 40.83056, -73.8508..."
3,YES,25,139.729472601,HEMPSTEAD AVENUE,"MULTILINESTRING ((-73.73489 40.71307, -73.7344..."
4,YES,25,280.118190658,14 AVENUE,"MULTILINESTRING ((-74.00523 40.61897, -74.0058..."
...,...,...,...,...,...
40068,YES,45,200.090878509,BROOKLYN QUEENS EXPRESSWAY,"MULTILINESTRING ((-73.99040 40.70165, -73.9896..."
40069,YES,25,222.224399088,FLUSHING AVENUE,"MULTILINESTRING ((-73.92286 40.70855, -73.9223..."
40070,YES,50,375.371745688,BELT PARKWAY,"MULTILINESTRING ((-73.88350 40.63227, -73.8835..."
40071,YES,25,436.5311759,OCEAN PARKWAY,"MULTILINESTRING ((-73.96760 40.58156, -73.9678..."
